# Overview

This notebook uses pyapacheatlas to inject lineage between input and output assets 

In [1]:
pip install pyapacheatlas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install snowflake-connector-python

     ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
     ---------------------------------------- 65.6/65.6 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/913.3 kB ? eta -:--:--
   -------------------------- ------------ 614.4/913.3 kB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 913.3/913.3 kB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/105.0 kB ? eta -:--:--
   ---------------------------------------- 105.0/105.0 kB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------- -------------------------------- 0.6/2.9 MB 17.3 MB/s eta 0:00:01
   ----------- ---------------------------- 0.8/2.9 MB 13.2 MB/s eta 0:00:01
   ---------------- ----------------------- 1.2/2.9 MB 9.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.7/2.9 MB 9.8 MB/s eta 0:00:01
   ------------------------------ --------- 2.2/2.9 MB 10.7 MB/s eta 0:00:01
   -----------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install pandas

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.4/11.5 MB 18.0 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/11.5 MB 12.4 MB/s eta 0:00:01
   --------- ------------------------------ 2.8/11.5 MB 10.5 MB/s eta 0:00:01
   ----------- ---------------------------- 3.3/11.5 MB 10.0 MB/s eta 0:00:01
   ------------- -------------------------- 4.0/11.5 MB 9.7 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.5 MB 8.4 MB/s eta 0:00:01
   ---------------- ----------------------- 4.9/11.5 MB 8.2 MB/s eta 0:00:01
   ------------------- -------------------- 5.5/11.5 MB 8.0 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.5 MB 7.4 MB/s eta 0:00:01
   -----


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
# Install Atlas Python client (https://github.com/wjohnson/pyapacheatlas)
import pyapacheatlas
import json
import snowflake.connector
import pandas as pd
import uuid

from pyapacheatlas.core import AtlasEntity
from pyapacheatlas.core import AtlasProcess
from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient
from pyapacheatlas.core.collections import PurviewCollectionsClient

In [34]:
# Define variable

# SPN + Purview account
vTenantID = ""
vClientID = ""
vClientSecret = ""
vPurviewAccount = "pdedemopurv"

#Snowflake information
vUser=""
vPassword=""
vAccount=""
vWarehouse=""
vDatabase="TASTY_BYTES_SAMPLE_DATA"
vSchema="RAW_POS"

In [35]:


# enter your service principal credentials here
oauth = ServicePrincipalAuthentication(
    tenant_id= vTenantID,
    client_id=vClientID,
    client_secret=vClientSecret
)


conn = snowflake.connector.connect(
    user=vUser,
    password=vPassword,
    account=vAccount,
    warehouse=vWarehouse,
    database=vDatabase,
    schema=vSchema
    )

# Instantiate PurviewClient, enter your Purview account name here
client = PurviewClient(
    account_name =  vPurviewAccount,
    authentication=oauth
)

collection = PurviewCollectionsClient(
    endpoint_url =  "https://" + vPurviewAccount +".purview.azure.com/",
    authentication=oauth
)


In [59]:
# Check if the table exists in Purview
def check_entity_if_exist(client, qualifiedName, typeName):
    entities = client.get_entity(qualifiedName=qualifiedName, typeName=typeName)
    return len(entities) > 0

In [67]:
## Need to change SQL Query to snowflake to recover all source/target information
results = conn.cursor().execute("select 1").fetchall()
for rec in results:
    print(rec)
    SourceQualifiedName= "snowflake://https/databases/SNOWFLAKE_SAMPLE_DATA/schemas/TPCDS_SF10TCL/tables/CALL_CENTER"
    TargetQualifiedName= "snowflake://https/databases/SNOWFLAKE_SAMPLE_DATA/schemas/TPCDS_SF100TCL/tables/CALL_CENTER"
    SourceTableName="CALL_CENTER"
    TargetTableName="CALL_CENTER"
    if check_entity_if_exist(client, SourceQualifiedName, "snowflake_table"):
        SourceLineage = AtlasEntity(
            name=SourceTableName,
            typeName="snowflake_table",
            qualified_name=SourceQualifiedName,
            #guid=str(uuid.uuid4())
        )
    if check_entity_if_exist(client, TargetQualifiedName, "snowflake_table"):
        TargetLineage = AtlasEntity(
            name=TargetTableName,
            typeName="snowflake_table",
            qualified_name=TargetQualifiedName,
            #guid=str(uuid.uuid4())
        )

    process_qn = "snowflake://sqlquery"
    process_type_name = "Process"
    vuuid = "-"+ str(uuid.uuid4())

    process = AtlasProcess(
        name="Snowflake query",
        typeName=process_type_name,
        qualified_name=process_qn,
        inputs=[SourceLineage],
        outputs=[TargetLineage],
        guid=vuuid
    )
    results = client.upload_entities(batch=[process])

(1,)
